In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
class RecipeRecommender:
    
    def __init__(self, ingreditans = [], sıradaki_besli = [0,5],):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.sırdaki_besli = sıradaki_besli
        self.ingreditans = ingreditans if ingreditans is not None else []
        self.df = pd.read_csv("RAW_recipes.csv")
    
    def benzer_yemekler(self):
        df = pd.read_csv("RAW_recipes.csv")
        tfidf = TfidfVectorizer()
        malzemeler_str = " ".join(self.ingreditans)
        malzemeler_vektoru = tfidf.fit_transform([malzemeler_str])
        cosbenzerlik = cosine_similarity(malzemeler_vektoru, tfidf.transform(self.df["ingredients"]))
        self.df["cosbenzerlik"] = cosbenzerlik[0]
        self.df = df.sort_values(by="cosbenzerlik", ascending=False)
        
    def sıradaki_yemekler(self):
        benzer_yemekler = df[self.sırdaki_besli[0]:self.sırdaki_besli[1]]
        while True:
            devam_mı = input("çıkış için : q, sırdaki beşli için 1")
            if devam_mı == 1:
                self.sırdaki_besli[0] +=5
                self.sırdaki_besli[1] +=5
                return benzer_yemekler
            else:
                break
    def içindekiler(self,yeni_yemek):
        self.ingreditans.append(yeni_yemek)

yemek = RecipeRecommender()
while True:
    islem = input("işlem seçiniz")
    if islem == "q":
        print("işlem sonlandırılıyore...")
        break
    elif islem == "1":
        yemek.içindekiler()
    elif islem == "4":
        yemek_ismi = input("yemek ismi giriniz")
        yemek_listesi = yemek_ismi.split(",")
        for i in yemek_listesi:
            yemek.içindekiler(i)
